In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
import glob
import pymongo
import pickle
from scipy.ndimage import uniform_filter1d
import os
from itertools import zip_longest
from matplotlib.lines import Line2D

In [2]:
#string lists for simulation filename ordering
chainstiff = ['flex']
allEEAAall = ['0.0','0.05','0.1','0.15','0.2','0.3','0.45','0.6']
allrbAAall = ['1.2','1.8','2.7','3.0','3.9','6.0']
Ea_constant = 5.309973197444597 #Value from activation energy calculation in "copoly5_activation_energy.ipynb"


#To find A for each epsilon, the result is stored in Amethod1
allslope = []
allsum2 = []
allslope2 = []
allA_method1 = []
trials = 5

for i in range(len(allEEAAall)):
    allsum = []
    allkprime = []
    for e in range(len(allrbAAall)):
        Ea = e
        Eaall = float(allrbAAall[e])/3 + Ea_constant
        #print(np.exp(-Eaall) + np.exp(-Eaall))
        summ = np.exp(-Eaall) + np.exp(-Eaall)
        allsum.append(np.exp(-Eaall) + np.exp(-Eaall))
        allXn = []
        alltstep = []
        for trial in range(1,trials+1): 


            simname = '220121_copoly_%sepsAA_%sepsAB_%sepsBB_%srbAA_%srbBB_%srbAB_%s-%i'%(allEEAAall[i],
                                                                                          allEEAAall[i],
                                                                                           allEEAAall[i],
                                                                                           allrbAAall[e],
                                                                                         allrbAAall[e],
                                                                                         allrbAAall[e],
                                                                                         chainstiff[0],trial)





            #print(simname)

            with open('../../alljarfile/'+simname+'_extent.pickle','rb') as f1:
                p = pickle.load(f1)
            with open('../../alljarfile/'+simname+'_tstep.pickle','rb') as f1:
                tstepeach = pickle.load(f1)
                alltstep.append(tstepeach)
            with open('../../alljarfile/'+simname+'_probs.pickle','rb') as f1:
                probs = pickle.load(f1)


            with open('../../alljarfile/'+simname+'_DOP.pickle','rb') as f1:
                Xneach = pickle.load(f1)
                allXn.append(Xneach)




        Xn = np.nanmean(np.array(list(zip_longest(*allXn)),dtype = float),axis = 1)
        tstep = np.nanmean(np.array(list(zip_longest(*alltstep)),dtype = float),axis = 1)


        slope, intercept, r_value, p_value, std_err = stats.linregress(tstep/1000,Xn)
        k = slope/(7200/(50*50*50))
        allkprime.append(k)
    slope, intercept, r_value, p_value, std_err = stats.linregress(allsum,allkprime)
    #print(simname,slope,r_value**2, allEEAAall[i])
    allA_method1.append(slope)





                   

In [3]:
#All A Method is the A values for the each epsilon value list in allEEAAall
print(allA_method1)


[0.6151745942359972, 0.6333451450392329, 0.6384711790706146, 0.6363153322843144, 0.6558098101358059, 0.6812190499375526, 0.697453078868466, 0.7483697879530885]


In [4]:
#Get standard deviation, output in allstd_error 
allslope = []

allsum2 = []
allslope2 = []
allA_method1 = []
trials = 5
allstdXn = []
standardk = []

allstd_error = []

for i in range(len(allEEAAall)):
    allsum = []
    allkprime = []
    for e in range(len(allrbAAall)):
        Ea = e
        allXn = []
        alltstep = []
        allk  = []
        for trial in range(1,trials+1): 
            simname = '220121_copoly_%sepsAA_%sepsAB_%sepsBB_%srbAA_%srbBB_%srbAB_%s-%i'%(allEEAAall[i],

                                                                                          allEEAAall[i],
                                                                                           allEEAAall[i],
                                                                                           allrbAAall[e],
                                                                                         allrbAAall[e],
                                                                                         allrbAAall[e],
                                                                                         chainstiff[0],trial)

            #print(simname)
            Eaall = float(allrbAAall[e])/3 + Ea_constant

            summ = np.exp(-Eaall) + np.exp(-Eaall)
            allsum.append(np.exp(-Eaall) + np.exp(-Eaall))

            with open('../../alljarfile/'+simname+'_extent.pickle','rb') as f1:
                p = pickle.load(f1)
            with open('../../alljarfile/'+simname+'_tstep.pickle','rb') as f1:
                tstepeach = pickle.load(f1)
                alltstep.append(tstepeach)
            with open('../../alljarfile/'+simname+'_probs.pickle','rb') as f1:
                probs = pickle.load(f1)
            with open('../../alljarfile/'+simname+'_DOP.pickle','rb') as f1:
                Xneach = pickle.load(f1)
                allXn.append(Xneach)

            slope, intercept, r_value, p_value, std_err = stats.linregress(tstepeach/1000,Xneach)
            k = slope/(7200/(50*50*50))
            allk.append(k)
            allkprime.append(k)
        standardk.append(np.mean(allk))
            
                
    slope, intercept, r_value, p_value, std_err = stats.linregress(allsum,allkprime)
    #print(simname,slope,r_value**2,std_err)
    allstd_error.append(std_err)
    



                   

In [5]:
print(allstd_error)

[0.00544321671179016, 0.00978287717868724, 0.006591302302070473, 0.007274848504310288, 0.007388299622405438, 0.010420641234491565, 0.008139359473214812, 0.009420500089548612]
